## Importing libraries

In [1]:
import os 
import re
import string

from tqdm.auto import tqdm
from razdel import sentenize

## Document desctiption

The document consists of three sections:
- the first section publishes the laws of the Republic of Crimea;
- in the second section — resolutions of the State Council of the Republic of Crimea;
- in the third section — resolutions of the Presidium of the State Council of the Republic Crimea.


All articles in second section and third section translated into three languages: Russian, Ukrainian and Crimean Tatar


- РАЗДЕЛ ВТОРОЙ (ru) - ЭКИНДЖИ БОЛЮК (crh) - РОЗДІЛ ДРУГИЙ (ua) (section two)
- РАЗДЕЛ ТРЕТИЙ (ru) - УЧЮНДЖИ БОЛЮК (crh) - РОЗДІЛ ТРЕТІЙ (ua) (section three)


## Split into language blocks

In [41]:
with open('data/txt_2/2015_1.txt', 'r', encoding='utf-8', errors='ignore') as document:
    lines = document.readlines()
    line_idxs_section2 = {}
    line_idxs_section3 = {'end':[len(lines)-1]}
    for idx, line in enumerate(lines):
        if line.startswith(('ЭКИНДЖИ БОЛЮК', 'Э КИНДЖИ БОЛЮК')): 
            if 'crh' not in line_idxs_section2:
                line_idxs_section2['crh'] = [idx]
            else: 
                line_idxs_section2['crh'].append(idx)
        elif line.startswith(('РАЗДЕЛ ВТОРОЙ', 'Р АЗДЕЛ ВТОРОЙ')):
            if 'ru' not in line_idxs_section2:
                line_idxs_section2['ru'] = [idx]
            else: 
                line_idxs_section2['ru'].append(idx)
        elif line.startswith(('РОЗДІЛ ДРУГИЙ', 'Р ОЗДІЛ ДРУГИЙ')):
            if 'ua' not in line_idxs_section2:
                line_idxs_section2['ua'] = [idx]
            else: 
                line_idxs_section2['ua'].append(idx)
        elif line.startswith(('РАЗДЕЛ ТРЕТИЙ', 'Р АЗДЕЛ ТРЕТИЙ')):
            if 'ru_3' not in line_idxs_section2:
                line_idxs_section2['ru_3'] = [idx]
            else: 
                line_idxs_section2['ru_3'].append(idx)
            if 'ru' not in line_idxs_section3: 
                line_idxs_section3['ru'] = [idx]
            else: 
                line_idxs_section3['ru'].append(idx)
        elif line.startswith(('РОЗДІЛ ТРЕТІЙ', 'Р ОЗДІЛ ТРЕТІЙ')):
            if 'ua' not in line_idxs_section3:
                line_idxs_section3['ua'] = [idx]
            else: 
                line_idxs_section3['ua'].append(idx)
        elif line.startswith(('УЧЮНДЖИ БОЛЮК', 'У ЧЮНДЖИ БОЛЮК')): 
            if 'crh' not in line_idxs_section3:
                line_idxs_section3['crh'] = [idx]
            else: 
                line_idxs_section3['crh'].append(idx)

In [42]:
line_idxs_section2

{'ru': [19, 1527], 'ru_3': [26, 1753], 'ua': [1604], 'crh': [1675]}

In [43]:
line_idxs_section3

{'end': [2230], 'ru': [26, 1753], 'ua': [2092], 'crh': [2158]}

In [44]:
idx_section2_sorted = sorted([max(line_idxs_section2[keys]) for keys in line_idxs_section2])
idx_section3_sorted = sorted([max(line_idxs_section3[keys]) for keys in line_idxs_section3])

In [45]:
crh_lines_section2 = lines[max(line_idxs_section2['crh']):idx_section2_sorted[idx_section2_sorted.index(max(line_idxs_section2['crh']))+1]]
ru_lines_section2 = lines[max(line_idxs_section2['ru']):idx_section2_sorted[idx_section2_sorted.index(max(line_idxs_section2['ru']))+1]]

In [46]:
crh_lines_section3 = lines[max(line_idxs_section3['crh']):idx_section3_sorted[idx_section3_sorted.index(max(line_idxs_section3['crh']))+1]]
ru_lines_section3 = lines[max(line_idxs_section3['ru']):idx_section3_sorted[idx_section3_sorted.index(max(line_idxs_section3['ru']))+1]]

The structure **includes** two types of articles:
- ПОСТАНОВЛЕНИЕ ГОСУДАРСТВЕННОГО СОВЕТА РЕСПУБЛИКИ КРЫМ (ru) - КЪЫРЫМ ДЖУМХУРИЕТИ ДЕВЛЕТ ШУРАСЫНЫНЪ КЪАРАРЫ (crh) (resolution of the State Council of the Republic of Crimea)
- ПОСТАНОВЛЕНИЕ ПРЕЗИДИУМА ГОСУДАРСТВЕННОГО СОВЕТА РЕСПУБЛИКИ КРЫМ (ru) - КЪЫРЫМ ДЖУМХУРИЕТИ ДЕВЛЕТ ШУРАСЫ ПРЕЗИДИУМЫНЫНЪ КЪАРАРЫ (crh) (resolution of the Presidium of the State Council of the Republic of Crimea)

Resolution of the **State Council** of the Republic of Crimea ends with frase:
- Председатель Государственного Совета Республики Крым (ru) - Къырым Джумхуриети Девлет Шурасынынъ Реиси (crh) (Chairman of the State Council of the Republic of Crimea)

Resolution of the **Presidium of the State Council** of the Republic of Crimea also ends with frase:
- Председатель Государственного Совета Республики Крым (ru) - Къырым Джумхуриети Девлет Шурасынынъ Реиси (crh) (Chairman of the State Council of the Republic of Crimea) 

Let's start with section 2: 

In [52]:
crh_idx_list_section2=[]
for idx, line in enumerate(crh_lines_section2):
    line = line.strip()
    if 'КЪЫРЫМ ДЖУМХУРИЕТИ' == line:
        crh_idx_list_section2.append([idx])
    if line.startswith('Къырым Джумхуриети Девлет Шурасынынъ Реиси'): 
        if len(crh_idx_list_section2) < 1: 
            continue
        elif len(crh_idx_list_section2[-1]) != 1:
            continue
        else: 
            crh_idx_list_section2[-1].append(idx)

In [53]:
crh_idx_list_section2

[[1, 30], [35, 74]]

In [54]:
ru_idx_list_section2=[]
for idx, line in enumerate(ru_lines_section2): 
    line = line.strip()
    if 'ПОСТАНОВЛЕНИЕ' == line:
        ru_idx_list_section2.append([idx])
    if line.startswith('Председатель Государственного Совета Республики Крым'): 
        if len(ru_idx_list_section2) < 1: 
            continue
        elif len(ru_idx_list_section2[-1]) != 1:
            continue
        else: 
            ru_idx_list_section2[-1].append(idx)

In [58]:
ru_idx_list_section2

[[1, 30], [35, 73]]

In [59]:
crh_idx_list_section2

[[1, 30], [35, 74]]

In [60]:
ru_lines_section2[1:30]

['ПОСТАНОВЛЕНИЕ  \n',
 'ГОСУДАРСТВЕННОГО СОВЕТА РЕСПУБЛИКИ КРЫМ\n',
 'ОБ ОБР АЗОВАНИИ СЕКРЕТАРИАТА ВНЕОЧЕРЕДНОЙ СЕССИИ  \n',
 'ГОСУДАРСТВЕННОГО СОВЕТА РЕСПУБЛИКИ КРЫМ ПЕРВОГО \n',
 'СОЗЫВА\n',
 'В соответствии со статьей 75 Конституции Республики Крым, статьей 44 Регламента Го -\n',
 'сударственного Совета Республики Крым, утвержденного Постановлением Государственного \n',
 'Совета Республики Крым от 24 декабря 2014 года № 376-1/14,\n',
 'Государственный Совет Республики Крым  п о с т а н о в л я е т:\n',
 '1. Установить численность секретариата внеочередной сессии Государственного Совета \n',
 'Республики Крым первого созыва в количестве    5 человек.\n',
 '2. Избрать секретариат внеочередной сессии Государственного Совета Республики Крым \n',
 'первого созыва в следующем составе:\n',
 'Георгиади Лариса Федоровна — депутат, избранный по республиканскому списку кан -\n',
 'дидатов Крымского регионального отделения Всероссийской политической партии «ЕДИНАЯ \n',
 'РОССИЯ»; \n',
 'Лантух 

In [61]:
crh_lines_section2[1:30]

['КЪЫРЫМ ДЖУМХУРИЕТИ \n',
 'ДЕВЛЕТ ШУР АСЫНЫНЪ КЪАР АРЫ\n',
 'БИРИНДЖИ ЧАГЪЫРЫШ КЪЫРЫМ ДЖУМХУРИЕТИ ДЕВЛЕТ  \n',
 'ШУР АСЫНЫНЪ НЕВБЕТТЕН ТЫШ СЕССИЯНЫНЪ КЯТИБЛИГИ -\n',
 'НИ ТЕШКИЛЬ ЭТИЛЬМЕСИ АКЪКЪЫНДА\n',
 'Къырым Джумхуриети Анаясасынынъ 75 маддесине, 2014 сенеси декабрь 24-те Къы -\n',
 'рым Джумхуриети Девлет Шурасынынъ 376-1/14 санлы Къарарынен тасдикълангъан Къырым \n',
 'Джумхуриети Девлет Шурасы Регламентининъ 21 маддесине мувафыкъ\n',
 'Къырым Джумхуриетининъ Девлет Шурасы  к ъ а р а р  б е р е: \n',
 '1. Биринджи чагъырыш Къырым Джумхуриети Девлет Шурасынынъ невбеттен тыш сес -\n',
 'сиянынъ кятиблиги 5 инсан микъдарында тайинленсин.\n',
 '2. Биринджи чагъырыш Къырым Джумхуриети Девлет Шурасынынъ невбеттен тыш сес -\n',
 'сиянынъ кятиблиги ашагъыдаки теркипте тешкиль олунсын:\n',
 'Георгиади Лариса Федоровна — «РУСИЕ БИРЛИГИ» Умумрусие сиясий фыркъасы \n',
 'Къырым джумхуриет болюгининъ джумхурий кандидатлар джедвели боюнджа сайлангъан \n',
 'депутаты; \n',
 'Лантух Наталья Анд

Working at section three: 

In [63]:
crh_idx_list_section3=[]
for idx, line in enumerate(crh_lines_section3):
    line = line.strip()
    if 'КЪЫРЫМ ДЖУМХУРИЕТИ' == line:
        crh_idx_list_section3.append([idx])
    if line.startswith('Къырым Джумхуриети Девлет Шурасынынъ Реиси'): 
        if len(crh_idx_list_section3) < 1: 
            continue
        elif len(crh_idx_list_section3[-1]) != 1:
            continue
        else: 
            crh_idx_list_section3[-1].append(idx)

In [64]:
crh_idx_list_section3

[[1, 16], [20, 42], [46, 68]]

In [65]:
ru_idx_list_section3=[]
for idx, line in enumerate(ru_lines_section3): 
    line = line.strip()
    if 'ПОСТАНОВЛЕНИЕ ПРЕЗИДИУМА' == line:
        ru_idx_list_section3.append([idx])
    if line.startswith('Председатель Государственного Совета Республики Крым'): 
        if len(ru_idx_list_section3) < 1: 
            continue
        elif len(ru_idx_list_section3[-1]) != 1:
            continue
        else: 
            ru_idx_list_section3[-1].append(idx)

In [66]:
ru_idx_list_section3

[[1, 16], [71, 87], [274, 295]]

## Text preprocessing

In [67]:
def clean_article(article: list[str]) -> str: 
    text = " ".join(line.strip() for line in article) 
    text = text.lower().replace('- ', '')
    matches = re.findall(r"((?:[а-яА-я]\s){3,}[а-яА-я])", text)
    for match in matches: 
        text = text.replace(match, "".join(match.split()))
    return text 

In [69]:
crh = clean_article(crh_lines_section3[1:16])
print(crh)

къырым джумхуриети девлет шур асы президиумынынъ къар ары къырымнынъ русиеге къошу лмасына бир йыл толма мунасебетинен кутьлеме азыр лыгъы акъкъында 2011 сенеси декабрь 29-да къабул олунгъан «къырым джумхуриетинде байрам ве ресмий куньлер акъкъында» 55-къджкъ/2014 санлы къырым джумхуриети къанунына, къырымнынъ русиеге къошулмасына бир йыл толма мунасебетинен кутьлеме азырлыгъы макъсадынен, къырым джумхуриети девлет шурасынынъ президиумы  къарар  бере: 1. къырымнынъ русиеге къошулмасына бир йыл толма мунасебетинен кутьлеме азыр лыгъы боюнджа тешкилий комитет къурулсын. 2. къырымнынъ русиеге къошулмасына бир йыл толма мунасебетинен кутьлеме азыр лыгъы боюнджа тешкилий комитетнинъ теркиби тасдикълансын (иляве этиле)*. 3. къырымнынъ русиеге къошулмасына бир йыл толма мунасебетинен кутьлеме азыр лыгъы боюнджа тешкилий комитет къырымнынъ русиеге къошулмасына бир йыл толма му насебетинен кутьлеме тедбир планы азырласын.


In [70]:
ru = clean_article(ru_lines_section3[1:16])
print(ru)

постановление президиума государственного совета республики крым о подготовке к пр азднованию первой годовщины воссоединения крыма с россией в соответствии с законом республики крым от 29 декабря 2014 года № 55-зрк/2014 «о праздниках и памятных датах в республике крым», с целью подготовки к празднованию первой годовщины воссоединения крыма с россией президиум государственного совета республики крым  постановляет: 1. образовать организационный комитет по подготовке к празднованию первой годовщи ны воссоединения крыма с россией. 2. утвердить состав организационного комитета по подготовке к празднованию первой годовщины воссоединения крыма с россией (прилагается). 3. организационному комитету по подготовке к празднованию первой годовщины воссо единения крыма с россией подготовить план мероприятий по празднованию первой годовщи ны воссоединения крыма с россией.


## Spliting into sub-paragraphs of the law

In [71]:
crh_sentences = [sent.strip() for sent in re.split('(?:\d{1,}\.)+', crh)]

In [72]:
crh_sentences

['къырым джумхуриети девлет шур асы президиумынынъ къар ары къырымнынъ русиеге къошу лмасына бир йыл толма мунасебетинен кутьлеме азыр лыгъы акъкъында 2011 сенеси декабрь 29-да къабул олунгъан «къырым джумхуриетинде байрам ве ресмий куньлер акъкъында» 55-къджкъ/2014 санлы къырым джумхуриети къанунына, къырымнынъ русиеге къошулмасына бир йыл толма мунасебетинен кутьлеме азырлыгъы макъсадынен, къырым джумхуриети девлет шурасынынъ президиумы  къарар  бере:',
 'къырымнынъ русиеге къошулмасына бир йыл толма мунасебетинен кутьлеме азыр лыгъы боюнджа тешкилий комитет къурулсын.',
 'къырымнынъ русиеге къошулмасына бир йыл толма мунасебетинен кутьлеме азыр лыгъы боюнджа тешкилий комитетнинъ теркиби тасдикълансын (иляве этиле)*.',
 'къырымнынъ русиеге къошулмасына бир йыл толма мунасебетинен кутьлеме азыр лыгъы боюнджа тешкилий комитет къырымнынъ русиеге къошулмасына бир йыл толма му насебетинен кутьлеме тедбир планы азырласын.']

In [73]:
ru_sentences = [sent.strip() for sent in re.split('(?:\d{1,}\.)+', ru)]

In [74]:
ru_sentences

['постановление президиума государственного совета республики крым о подготовке к пр азднованию первой годовщины воссоединения крыма с россией в соответствии с законом республики крым от 29 декабря 2014 года № 55-зрк/2014 «о праздниках и памятных датах в республике крым», с целью подготовки к празднованию первой годовщины воссоединения крыма с россией президиум государственного совета республики крым  постановляет:',
 'образовать организационный комитет по подготовке к празднованию первой годовщи ны воссоединения крыма с россией.',
 'утвердить состав организационного комитета по подготовке к празднованию первой годовщины воссоединения крыма с россией (прилагается).',
 'организационному комитету по подготовке к празднованию первой годовщины воссо единения крыма с россией подготовить план мероприятий по празднованию первой годовщи ны воссоединения крыма с россией.']

---

Some sub-paragraphs of the law could be splited into sentences. We use for this purpose razdel from [Natasha](https://github.com/natasha/razdel).

In [75]:
crh_sentences = list(sentenize(crh))
ru_sentences = list(sentenize(ru))

In [77]:
for s1, s2 in zip(crh_sentences, ru_sentences):
    print(s1.text)
    print()
    print(s2.text)
    print()
    print()

къырым джумхуриети девлет шур асы президиумынынъ къар ары къырымнынъ русиеге къошу лмасына бир йыл толма мунасебетинен кутьлеме азыр лыгъы акъкъында 2011 сенеси декабрь 29-да къабул олунгъан «къырым джумхуриетинде байрам ве ресмий куньлер акъкъында» 55-къджкъ/2014 санлы къырым джумхуриети къанунына, къырымнынъ русиеге къошулмасына бир йыл толма мунасебетинен кутьлеме азырлыгъы макъсадынен, къырым джумхуриети девлет шурасынынъ президиумы  къарар  бере: 1. къырымнынъ русиеге къошулмасына бир йыл толма мунасебетинен кутьлеме азыр лыгъы боюнджа тешкилий комитет къурулсын.

постановление президиума государственного совета республики крым о подготовке к пр азднованию первой годовщины воссоединения крыма с россией в соответствии с законом республики крым от 29 декабря 2014 года № 55-зрк/2014 «о праздниках и памятных датах в республике крым», с целью подготовки к празднованию первой годовщины воссоединения крыма с россией президиум государственного совета республики крым  постановляет: 1. обра

___